<a href="https://colab.research.google.com/github/BATiger/Classic/blob/main/YOLO_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Non-maximum suppression algorithm
import numpy as np

def py_cpu_nms(dets, thresh):
    """Pure Python NMS baseline."""
    #x1、y1、x2、y2、以及score赋值
    x1 = dets[:, 0]                     # pred bbox top_x
    y1 = dets[:, 1]                     # pred bbox top_y
    x2 = dets[:, 2]                     # pred bbox bottom_x
    y2 = dets[:, 3]                     # pred bbox bottom_y
    scores = dets[:, 4]              # pred bbox cls score
    #每一个检测框的面积
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)    # pred bbox areas
    print(areas)
    # 从大到小
    order = scores.argsort()[::-1]              # 对pred bbox按score做降序排序，对应step-2
    print(order)
    #保留的结果框集合
    keep = []    
    while order.size > 0:
        i = order[0]          # top-1 score bbox
        keep.append(i)   #保留该类剩余box中得分最高的一个
        #得到相交区域,左上及右下
        xx1 = np.maximum(x1[i], x1[order[1:]])   # top-1 bbox（score最大）与order中剩余bbox计算NMS
        print(xx1)
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])
        
        #计算相交的面积,不重叠时面积为0
        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        #计算IoU：重叠面积 /（面积1+面积2-重叠面积）
        ovr = inter / (areas[i] + areas[order[1:]] - inter)
        #保留IoU小于阈值的box
        inds = np.where(ovr <= thresh)[0]     # 这个操作可以对代码断点调试理解下，结合step-3，我们希望剔除所有与当前top-1 bbox IoU > thresh的冗余bbox，那么保留下来的bbox，自然就是ovr <= thresh的非冗余bbox，其inds保留下来，作进一步筛选
        #因为ovr数组的长度比order数组少一个，所以这里要将所有下标后移一位
        order = order[inds + 1]   # 保留有效bbox，就是这轮NMS未被抑制掉的幸运儿，为什么 + 1？因为ind = 0就是这轮NMS的top-1，剩余有效bbox在IoU计算中与top-1做的计算，inds对应回原数组，自然要做 +1 的映射，接下来就是step-4的循环

    return keep    # 最终NMS结果返回

dets = np.array([[100,120,170,200,0.98],
                  [20,40,80,90,0.99],
                  [20,38,82,88,0.96],
                  [200,380,282,488,0.9],
                  [19,38,75,91, 0.8]])

py_cpu_nms(dets, 0.5)


[5751. 3111. 3213. 9047. 3078.]
[1 0 2 3 4]
[100.  20. 200.  20.]
[200.]
[]


[1, 0, 3]